## TODO
 
[-] Remove the primary columns from evaluation  
[-] Make to top number variable based on inicial class input  
[-] Insert a button the follow the page to return to top  
[ ] Reagenge report render to a unique function/class  
[ ] Test using a continuos target  
[ ] Test using categorical variables  


### Import Data


In [1]:
import numpy as np
import pandas as pd

from utils import SandEDA

df = pd.read_csv("./database/UCI_Credit_Card.csv")

# Define the specific dates range
start_date = "2023-06-01"
end_date = "2023-09-30"

safra = []
# Generate a random date within the specified range
for i in range(df.shape[0]):
    safra.append(
        pd.to_datetime(
            np.random.choice(pd.date_range(start=start_date, end=end_date))
        ).strftime("%Y%m")
    )

df["safra"] = safra
df.rename(
    columns={
        "default.payment.next.month": "default",
    },
    inplace=True,
)


In [3]:
import numpy as np
import pandas as pd
import altair as alt
from jinja2 import Template


from utils import SandEDA

df = pd.read_csv("./database/Playground_Churn_Bank.csv")

del df['id']

# Define the specific dates range
start_date = "2023-06-01"
end_date = "2023-09-30"

safra = []
# Generate a random date within the specified range
for i in range(df.shape[0]):
    safra.append(
        pd.to_datetime(
            np.random.choice(pd.date_range(start=start_date, end=end_date))
        ).strftime("%Y%m")
    )

df["safra"] = safra

In [4]:
sandeda = SandEDA(df, "Exited", "safra", "CustomerId")
sandeda.report('CHURN_0_0_0')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifie["descriptive_statistics"]
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Define the features and target
X = df.drop(columns=['default', 'ID', 'safra'])
y = df['default']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the classifiers
classifiers = {
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "Logistic Regression": LogisticRegression(max_iter=1000)
}

# Train and evaluate each classifier
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy:.4f}")

Decision Tree Accuracy: 0.7202
Random Forest Accuracy: 0.8137


/home/lucas/Projects/janestreet/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:22:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.8167
Logistic Regression Accuracy: 0.8092


/home/lucas/Projects/janestreet/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
### OVERVxIEW

sandeda = SandEDA(df, "Exited", "safra", "CustomerId")
res_general = sandeda.calc_general()

overview_target_metric_time = res_general["target_general"]["target_metric_time"]

# Create the bar plot using Altair
chart = (
    alt.Chart(
        pd.DataFrame(
            {
                "Date": list(overview_target_metric_time.keys()),
                "%": list(overview_target_metric_time.values()),
            }
        )
    )
    .mark_bar()
    .encode(x=alt.X("Date", sort="ascending"), y=alt.Y("%"))
    .properties(width=100, height=100)
    .configure_axis(labelAngle=45)
    .configure_title(fontSize=10)
)


overview_tab_general = pd.DataFrame(
                        list(res_general["dataset_general"].items()),
                        columns=["description", "value"],
                        index=None,
                        ).to_html(index=False, border=0)

overview_tab_full = pd.DataFrame(res_general["missing_zero"]).to_html(
                        index=False, 
                        border=0)

overview_target_name = res_general["target_general"]["target_name"]

overview_target_metric = (
    res_general["target_general"]["number_of_one"]
    / (
        res_general["target_general"]["number_of_zero"]
        + res_general["target_general"]["number_of_one"]
    )
) * 100


overview_tgt_graph_json = chart.to_json()


### VARIABLES

iv_, mi_ = sandeda.promising_features()
psi_, ks_ = sandeda.variables_estability()
miss_, zero_ = sandeda.variables_fillment()

var_tab_ks = pd.DataFrame({
    'Variable': [var for var, _ in ks_],
    'Max KS': [max(value.values()) for _, value in ks_]
}).to_html(index=False, border=0)

var_tab_psi = pd.DataFrame({
    'Variable': [var for var, _ in psi_],
    'Max PSI': [max(value.values()) for _, value in psi_]
}).to_html(index=False, border=0)


var_tab_iv = pd.DataFrame({
    'Variable': [var for var, _ in iv_],
    'IV': [value for _, value in iv_]
}).to_html(index=False, border=0)

var_tab_mi = pd.DataFrame({
    'Variable': [var for var, _ in mi_],
    'MI': [value for _, value in mi_]
}).to_html(index=False, border=0)


var_tab_miss = pd.DataFrame(miss_).to_html(index=False, border=0)
var_tab_zero = pd.DataFrame(zero_).to_html(index=False, border=0)


### VARIABLES ESPECIFICS

variables_espec = sandeda.variables_espec()

variables_espec_time = sandeda.variables_espec_time()


var_espec_content = {}

vars_keys = variables_espec.keys() - {"Exited", "safra", "CustomerId"}

for var_espec in vars_keys:

    hist_var = variables_espec[var_espec]['histogram']

    decil_var = variables_espec[var_espec]['decil']

    del variables_espec[var_espec]['histogram'], variables_espec[var_espec]['decil']

    var_spec_tab_desc = pd.DataFrame(
        {
            "description": variables_espec[var_espec]["descriptive_statistics"].keys(),
            "value": variables_espec[var_espec]["descriptive_statistics"].values(),
        }).to_html(index=False, border=0)


    var_spec_tab_quant = pd.DataFrame(
        {
            "description": variables_espec[var_espec]["quantile_statistics"].keys(),
            "value": variables_espec[var_espec]["quantile_statistics"].values(),
        }).to_html(index=False, border=0)

    if variables_espec[var_espec]["descriptive_statistics"]["number_of_unique_values"] <= 50 or variables_espec[var_espec]["descriptive_statistics"]['variable_type'] not in ["int64", "float64"]:
        # Convert the histogram data to a dataframe
        hist_data = pd.DataFrame({
            'Interval': list(hist_var.keys()),
            'Count': list(hist_var.values())
        })
    else:
        hist_data = pd.DataFrame({
            'Interval': [f"{interval.left:.1f}" for interval in hist_var.keys()],
            'Count': list(hist_var.values())
        })

    # Create the bar plot using Altair
    hist_chart = alt.Chart(hist_data).mark_bar().encode(
        x=alt.X('Interval', title='', sort=None, axis=alt.Axis(labels=True, labelOverlap=True, labelFontSize=9)),
        y=alt.Y('Count', title='Qty', axis=alt.Axis(labels=False))
    ).properties(
        width=350,
        height=200,
        title='Histogram'
    ).configure_axis(
        labelAngle=45
    ).configure_title(
        fontSize=14
    ).interactive(False)  # Disable interactive features

    var_spec_hist_graph_json = hist_chart.to_json()

    decil_data = pd.DataFrame({
        'Decil': list(decil_var.index),
        'Target': list(decil_var.values)
    })

    # Create the bar plot using Altair
    decil_chart = alt.Chart(decil_data).mark_bar().encode(
        x=alt.X('Decil', title='Decil', sort=None, axis=alt.Axis(labels=True, labelOverlap=True, labelFontSize=9)),
        y=alt.Y('Target', title='% Target', axis=alt.Axis(labels=False))
    ).properties(
        width=350,
        height=200,
        title='Target mean per Decil'
    ).configure_axis(
        labelAngle=45
    ).configure_title(
        fontSize=12
    ).interactive(False)  # Disable interactive features

    var_spec_decil_graph_json = decil_chart.to_json()

    # Create a dataframe with the number_per_quintile values for each month
    df_quintile_time = pd.DataFrame({
        'Date': list(variables_espec_time[var_espec].keys()),
        'number_per_quintile': [variables_espec_time[var_espec][date]['number_per_quintile'] for date in variables_espec_time[var_espec].keys()]
    })

    # Expand the number_per_quintile dictionary into separate columns
    df_quintile_time = df_quintile_time.join(pd.DataFrame(df_quintile_time.pop('number_per_quintile').tolist(), index=df_quintile_time.index))

    # Melt the dataframe to have a long format suitable for Altair
    df_melted = df_quintile_time.melt(id_vars='Date', var_name='Quintile', value_name='Count')

    # Calculate the percentage for each quintile
    df_melted['Percentage'] = df_melted.groupby('Date')['Count'].transform(lambda x: x / x.sum() * 100)

    # Create the 100% stacked column chart using Altair
    stacked_chart = alt.Chart(df_melted).mark_bar().encode(
        x=alt.X('Date', title='Date'),
        y=alt.Y('Percentage', title='Percentage', stack='normalize'),
        color=alt.Color('Quintile', title='Quintile')
    ).properties(
        width=800,
        height=200,
        title='100% Stacked Column Chart'
    ).configure_axis(
        labelAngle=45
    ).configure_title(
        fontSize=14
    ).configure_legend(
        orient='top'
    ).interactive(False)  # Disable interactive features

    var_spec_stacked_graph_json = stacked_chart.to_json()

    var_espec_content[var_espec] = {
        "tab_desc": var_spec_tab_desc,
        "tab_quant": var_spec_tab_quant,
        "hist": var_spec_hist_graph_json,
        "decil": var_spec_decil_graph_json,
        "hist_time": var_spec_stacked_graph_json,
    }

### JINJA TEMPLATE

# Read the template from the template sheet
with open("./template/sandEda_template.html", "r") as file:
    sanEda_template = file.read()
    
# Create a Jinja2 template object

template = Template(sanEda_template)

# Render the template with the data
rendered_html = template.render(
    overview_tab_general=overview_tab_general,
    overview_tab_full=overview_tab_full,
    overview_target_metric=overview_target_metric,
    overview_target_name=overview_target_name,
    overview_tgt_graph_json=overview_tgt_graph_json,
    var_tab_iv=var_tab_iv,
    var_tab_mi=var_tab_mi,
    var_tab_miss=var_tab_miss,
    var_tab_zero=var_tab_zero,
    var_tab_psi=var_tab_psi,
    var_tab_ks=var_tab_ks,
    specific_variables=var_espec_content,
)

# Save the rendered HTML to a file
with open("sanEda_report.html", "w") as file:
    file.write(rendered_html)


In [ ]:
from sklearn.preprocessing import LabelEncoder

time_name = "safra"
target_name = "Exited"
id_name = "CustomerId"

res_espect_var = {}
quintil_cut_points = {}

df_filtered = df.drop(
    columns=[time_name, target_name, id_name]
)

for var in df_filtered:
    res_espect_time = {}
    for time in df[time_name].unique():
        df_time = df[df[time_name] == time]

        res_espec = {}

        if df_time[var].dtypes == "object":
            le = LabelEncoder()
            df_time_enc[var] = le.fit_transform(df_time[var])

            quintil_cut_points[var] = (
                df[var].quantile([0.2, 0.4, 0.6, 0.8]).to_dict()
            )

            res_espec = {
                "number_of_missing": int(df_time[var].isnull().sum()),
                "number_of_zeros": int((df_time[var] == 0).sum()),
                "sum": float(df_time[var].sum()),
                "mean": float(round(df_time[var].mean(), 2)),
                "median": float(df_time[var].median()),
                "number_per_quintile": {
                    f"<= {round(quintil_cut_points[var][0.2], 0)}": int(
                        (df_time[var] <= quintil_cut_points[var][0.2]).sum()
                    ),
                    f"<= {round(quintil_cut_points[var][0.4], 0)}": int(
                        (
                            (df_time[var] > quintil_cut_points[var][0.2])
                            & (df_time[var] <= quintil_cut_points[var][0.4])
                        ).sum()
                    ),
                    f"<= {round(quintil_cut_points[var][0.6], 0)}": int(
                        (
                            (df_time[var] > quintil_cut_points[var][0.4])
                            & (df_time[var] <= quintil_cut_points[var][0.6])
                        ).sum()
                    ),
                    f"<= {round(quintil_cut_points[var][0.8], 0)}": int(
                        (
                            (df_time[var] > quintil_cut_points[var][0.6])
                            & (df_time[var] <= quintil_cut_points[var][0.8])
                        ).sum()
                    ),
                    f"> {round(quintil_cut_points[var][0.8], 0)}": int(
                        (df_time[var] > quintil_cut_points[var][0.8]).sum()
                    ),
                },
            }
        elif df_time[var].dtype in ["int64", "float64"]:
            quintil_cut_points[var] = (
                df[var].quantile([0.2, 0.4, 0.6, 0.8]).to_dict()
            )
            res_espec = {
                "number_of_missing": int(df_time[var].isnull().sum()),
                "number_of_zeros": int((df_time[var] == 0).sum()),
                "sum": float(df_time[var].sum()),
                "mean": float(round(df_time[var].mean(), 2)),
                "median": float(df_time[var].median()),
                "number_per_quintile": {
                    f"<= {round(quintil_cut_points[var][0.2], 0)}": int(
                        (df_time[var] <= quintil_cut_points[var][0.2]).sum()
                    ),
                    f"<= {round(quintil_cut_points[var][0.4], 0)}": int(
                        (
                            (df_time[var] > quintil_cut_points[var][0.2])
                            & (df_time[var] <= quintil_cut_points[var][0.4])
                        ).sum()
                    ),
                    f"<= {round(quintil_cut_points[var][0.6], 0)}": int(
                        (
                            (df_time[var] > quintil_cut_points[var][0.4])
                            & (df_time[var] <= quintil_cut_points[var][0.6])
                        ).sum()
                    ),
                    f"<= {round(quintil_cut_points[var][0.8], 0)}": int(
                        (
                            (df_time[var] > quintil_cut_points[var][0.6])
                            & (df_time[var] <= quintil_cut_points[var][0.8])
                        ).sum()
                    ),
                    f"> {round(quintil_cut_points[var][0.8], 0)}": int(
                        (df_time[var] > quintil_cut_points[var][0.8]).sum()
                    ),
                },
            }
        res_espect_time[time] = res_espec
    res_espect_var[var] = res_espect_time